#  Practica de limpieza de texto con pandas - Solución

#### Por Jose R. Zapata - https://joserzapata.github.io/


> **Objetivo**: practicar métodos de `pandas.Series.str` para limpiar y normalizar texto en DataFrames.
> Cada ejercicio incluye un **Enunciado** y una celda donde se ve el resultado de la solución.



**Referencias:**
- Procesamiento básico de texto (curso de NLP): https://joserzapata.github.io/courses/nlp/procesamiento-basico/
- Documentación de `pandas.Series.str`: https://pandas.pydata.org/docs/reference/series.html#string-handling


## Instalación de dependencias

instala e importa las librerias que necesites para ejecutar los ejercicios, por ejemplo para instalar Jupyter usa:

```bash
uv add jupyter
```


In [ ]:
# Copie su código aca


In [ ]:
import pandas as pd
import unicodedata
import re, numpy as np

## Conjuntos de datos de juguete
Creamos 4 DataFrames para practicar distintas tareas de limpieza: texto general, tweets, productos y personas.

In [ ]:
## Texto general
df_texto = pd.DataFrame(
    {
        "texto": [
            "  ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)  ",
            "Pandas > numpy? 🤔  Email: persona@example.com   ",
            "Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan",
            "Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123",
            "      TABLAS\t, \n espacios   y saltos de línea.\r\n",
            "Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44",
            "Dirección: Cll 10 # 5-20; Medellín. Barrio: La América",
            "Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",
        ]
    }
)

## Tweets
df_tweets = pd.DataFrame(
    {
        "tweet": [
            "RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python",
            "¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊",
            "Probando cosas en Jupyter... sin link ni hashtag",
            "@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml",
            "¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data",
        ]
    }
)

## Productos y precios
df_productos = pd.DataFrame(
    {
        "producto": [
            "Camisa talla M",
            "Pantalón-XL",
            "Zapato, Talla: 42",
            "Blusa s",
            "Medias 10-12",
            "Polo Talla l",
            "Vestido - 36",
            "Sombrero (talla Única)",
        ],
        "precio": [
            "$1.234,50",
            "USD 45",
            "30,00 €",
            "25.000",
            "$ 0",
            "S/. 120.90",
            "COP 9.990",
            "AR$ 2.550,00",
        ],
    }
)

## Personas, respuestas y direcciones
df_personas = pd.DataFrame(
    {
        "nombre": [
            "ana María LOPEZ",
            "Juan  perez",
            "Ñandú   Gómez",
            "Miguel (Soporte)",
            "  MÓNICA de la CRUZ  ",
            "luis-delgado",
        ],
        "categoria": ["Sí", "si", "SI ", "No", "—", None],
        "direccion": [
            "Calle 45 # 12-34, Bogotá",
            "Av. Siempre Viva 742 - Lima",
            "Cll. 10 No. 5-20 Medellín",
            "Cra 7a # 45-60, Bogotá",
            "Av. 9 #12-34  Cali",
            "Av. Insurgentes Sur 1234, CDMX",
        ],
        "id_raw": [
            "abc-0001",
            "abc 001",
            "ABC0002",
            "Abc_003",
            "abc-00004",
            "ABC-0005",
        ],
    }
)

df_texto:


,texto
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :)
1,Pandas > numpy? 🤔 Email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123
4,"TABLAS\t, \n espacios y saltos de línea.\r\n"
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"



df_tweets:


,tweet
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊
2,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data



df_productos:


,producto,precio
0,Camisa talla M,"$1.234,50"
1,Pantalón-XL,USD 45
2,"Zapato, Talla: 42","30,00 €"
3,Blusa s,25.000
4,Medias 10-12,$ 0
5,Polo Talla l,S/. 120.90
6,Vestido - 36,COP 9.990
7,Sombrero (talla Única),"AR$ 2.550,00"



df_personas:


,nombre,categoria,direccion,id_raw
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005


## Inspección rápida

- Explora los cuatro DataFrames
- Identifica qué columnas requieren **limpieza textual** y por qué (espacios, acentos, emojis, URLs, etc.).
- Escribe un breve comentario (como comentario en la celda) con tus observaciones.

In [5]:
# 1. DataFrame de texto general (df_texto)
print("--- df_texto ---")
display(df_texto)
# Observaciones df_texto['texto']:
# - Espacios extra al inicio, final e intermedios.
# - Mayúsculas y minúsculas mezcladas.
# - Signos de puntuación (!!!, :, ?, ;).
# - URLs (https://miweb.com).
# - Hashtags (#DataScience).
# - Emojis (:), 🤔, 😀).
# - Caracteres especiales como saltos de línea (\n, \r\n) y tabulaciones (\t).
# - Números de teléfono y direcciones.

# 2. DataFrame de tweets (df_tweets)
print("\n--- df_tweets ---")
display(df_tweets)
# Observaciones df_tweets['tweet']:
# - Prefijos de Retweet ("RT @...").
# - Menciones de usuario ("@...").
# - Hashtags ("#...").
# - URLs.
# - Emojis.
# - Mezcla de mayúsculas y minúsculas.

# 3. DataFrame de productos (df_productos)
print("\n--- df_productos ---")
display(df_productos)
# Observaciones df_productos:
# - 'producto': Formato de talla inconsistente ("talla M", "-XL", "Talla: 42", "s").
# - 'precio': Contiene símbolos de moneda ($, €, S/.), separadores de miles y decimales inconsistentes, y texto ("USD", "COP"). Necesita ser convertido a un tipo numérico.

# 4. DataFrame de personas (df_personas)
print("\n--- df_personas ---")
display(df_personas)
# Observaciones df_personas:
# - 'nombre': Mayúsculas/minúsculas, espacios extra, caracteres especiales ("()", "-") y tildes/eñes.
# - 'categoria': Respuestas categóricas inconsistentes ("Sí", "si", "SI "), guiones y valores nulos. Debería ser una columna booleana.
# - 'direccion': Formatos de dirección variados. Extraer la ciudad podría ser útil.
# - 'id_raw': IDs con formatos inconsistentes (guiones, espacios, mayúsculas).

--- df_texto ---


,texto
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...
1,Pandas > numpy? 🤔 Email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...
4,"TABLAS\t, \n espacios y saltos de líne..."
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"



--- df_tweets ---


,tweet
0,RT @maria: Nuevo post en el blog -> http://blo...
1,¡Me encanta Pandas! #datos #Python https://exa...
2,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://...
4,¿Pandas o Polars? debátanlo aquí 👉 https://for...



--- df_productos ---


,producto,precio
0,Camisa talla M,"$1.234,50"
1,Pantalón-XL,USD 45
2,"Zapato, Talla: 42","30,00 €"
3,Blusa s,25.000
4,Medias 10-12,$ 0
5,Polo Talla l,S/. 120.90
6,Vestido - 36,COP 9.990
7,Sombrero (talla Única),"AR$ 2.550,00"



--- df_personas ---


,nombre,categoria,direccion,id_raw
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   8 non-null      object
dtypes: object(1)
memory usage: 196.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   producto  8 non-null      object
 1   precio    8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     6 n

## 1) Normaliza a minúsculas

En el DataFrame `df_texto`, crea una nueva columna `texto_min` que contenga el texto de la columna `texto` en minúsculas.

In [ ]:
df_texto["texto_min"] = df_texto["texto"].str.lower()
df_texto[["texto", "texto_min"]]

,texto,texto_min
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,¡hola mundo!!! esto es un ejemplo: visita...
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...,me gusta el café colombiano; es buenísimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,oferta!!! 3x2 en jabón líquido 500ml - código:...
4,"TABLAS\t, \n espacios y saltos de líne...","tablas\t, \n espacios y saltos de líne..."
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,teléfono: (57) 300-123-45-67; whatsapp +57 300...
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,dirección: cll 10 # 5-20; medellín. barrio: la...
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


,texto,texto_min
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :)
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta!!! 3x2 en jabón líquido 500ml - código: a-123
4,"TABLAS\t, \n espacios y saltos de línea.\r\n","tablas\t, \n espacios y saltos de línea.\r\n"
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,dirección: cll 10 # 5-20; medellín. barrio: la américa
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 2) Eliminar Espacios

Genera  la columna `texto_espacios` eliminando espacios al inicio/fin

In [ ]:
df_texto["texto_espacios"] = df_texto["texto"].str.strip()
df_texto[["texto", "texto_espacios"]]

,texto,texto_espacios
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,¡Hola Mundo!!! Esto es un EJEMPLO: visita h...
1,Pandas > numpy? 🤔 Email: persona@example.com,Pandas > numpy? 🤔 Email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...,Me gusta el café colombiano; es buenísimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...
4,"TABLAS\t, \n espacios y saltos de líne...","TABLAS\t, \n espacios y saltos de línea."
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


,texto_espacios,texto_min
0,¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :),¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :)
1,pandas > numpy? 🤔 email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan
3,oferta!!! 3x2 en jabón líquido 500ml - código: a-123,oferta!!! 3x2 en jabón líquido 500ml - código: a-123
4,"tablas\t, \n espacios y saltos de línea.","tablas\t, \n espacios y saltos de línea.\r\n"
5,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44
6,dirección: cll 10 # 5-20; medellín. barrio: la américa,dirección: cll 10 # 5-20; medellín. barrio: la américa
7,"emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 3) Puntuación

Crea `texto_sin_punct` removiendo puntuación y símbolos, conservando letras y espacios.

In [20]:
df_texto["texto_sin_punct"] = (
    df_texto.get("tescto", df_texto["texto"])
    .astype(str)
    .str.lower()
    .map(
        lambda s: "".join(
            ch
            for ch in unicodedata.normalize("NFD", s)
            if ch.isspace() or unicodedata.category(ch)[0] in ("L", "N")
        )
    )
)
print(df_texto[["texto", "texto_sin_punct"]].to_string())


                                                                                  texto                                                           texto_sin_punct
0    ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)      hola mundo  esto es   un ejemplo visita httpsmiwebcom  datascience    
1                                      Pandas > numpy? 🤔  Email: persona@example.com                                   pandas  numpy   email personaexamplecom   
2                    Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan               me gusta el cafe colombiano es buenisimo cafe colombia juan
3                                  Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123                            oferta 3x2 en jabon liquido 500ml  codigo a123
4                                        TABLAS\t, \n espacios   y saltos de línea.\r\n                              tablas\t \n espacios   y saltos de linea\r\n
5                           

## 4) Acentos

Elimina los acentos de las vocales del texto `df_texto` y grábalo en una nueva columna `texto_sin_acentos`.

In [21]:
# Copie su código aca
df_texto['texto_sin_acentos'] = df_texto['texto' if 'texto' in df_texto.columns else 'tescto'].astype(str).str.translate(str.maketrans({
    'á':'a','é':'e','í':'i','ó':'o','ú':'u',
    'Á':'A','É':'E','Í':'I','Ó':'O','Ú':'U',
    'ä':'a','ë':'e','ï':'i','ö':'o','ü':'u',
    'Ä':'A','Ë':'E','Ï':'I','Ö':'O','Ü':'U',
    'à':'a','è':'e','ì':'i','ò':'o','ù':'u',
    'À':'A','È':'E','Ì':'I','Ò':'O','Ù':'U',
    'â':'a','ê':'e','î':'i','ô':'o','û':'u',
    'Â':'A','Ê':'E','Î':'I','Ô':'O','Û':'U',
    'ã':'a','õ':'o','Ã':'A','Õ':'O'
}))
print(df_texto[['texto', 'texto_sin_acentos']].to_string())

                                                                                  texto                                                                     texto_sin_acentos
0    ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)      ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)  
1                                      Pandas > numpy? 🤔  Email: persona@example.com                                         Pandas > numpy? 🤔  Email: persona@example.com   
2                    Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan                    Me gusta el cafe colombiano; es buenisimo!!! #Cafe #Colombia @juan
3                                  Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123                                  Oferta!!! 3x2 en jabon liquido 500ml - CODIGO: A-123
4                                        TABLAS\t, \n espacios   y saltos de línea.\r\n                                        TAB

0    hola mundo  esto es   un ejemplo visita httpsmiwebcom  datascience  
1                                 pandas  numpy   email personaexamplecom
2             me gusta el cafe colombiano es buenisimo cafe colombia juan
3                          oferta 3x2 en jabon liquido 500ml  codigo a123
4                                tablas\t \n espacios   y saltos de linea
5                        telefono 57 3001234567 whatsapp 57 300 222 33 44
6                        direccion cll 10  520 medellin barrio la america
7                                           emoji test  simbolos  y otros
Name: texto_sin_acentos, dtype: object

## 5) Emojis

Crear la columna `texto_sin_emoji` quitando emojis, pictogramas y símbolos gráficos comunes.

In [22]:
# Copie su código aca
# Columna base (usa 'texto' o cae a 'tescto' si fue el nombre usado antes)
base = "texto" if "texto" in df_texto.columns else "tescto"

# Crear columna sin emojis / pictogramas / símbolos gráficos (categoría Unicode 'S')
df_texto["texto_sin_emoji"] = (
    df_texto[base]
    .astype(str)
    .map(
        lambda s: "".join(
            ch
            for ch in s
            if unicodedata.category(ch)[0] != "S"
            and ch not in "\u200d\u200c\ufe0f\ufe0e"  # ZWJ/ZWNJ/VS
        )
    )
)

# Imprimir resultados (original + nueva columna)
print(df_texto[[base, "texto_sin_emoji"]].to_string())

                                                                                  texto                                                                       texto_sin_emoji
0    ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)      ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)  
1                                      Pandas > numpy? 🤔  Email: persona@example.com                                           Pandas  numpy?   Email: persona@example.com   
2                    Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan                    Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan
3                                  Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123                                  Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123
4                                        TABLAS\t, \n espacios   y saltos de línea.\r\n                                        TAB

0    ¡hola mundo!!!  esto es   un ejemplo: visita https://miweb.com  #datascience  :)
1                                        pandas > numpy?   email: persona@example.com
2                  me gusta el café colombiano; es buenísimo!!! #café #colombia @juan
3                                oferta!!! 3x2 en jabón líquido 500ml - código: a-123
4                                          tablas\t, \n espacios   y saltos de línea.
5                            teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44
6                              dirección: cll 10 # 5-20; medellín. barrio: la américa
7                                               emoji test: ️‍, símbolos ©®™ y otros…
Name: texto_sin_emoji, dtype: object

## 6) Extrae emails y URLs

En `df_texto`, crea las columnas `email` y `urls` extrayendo emails y URLs del texto original.

**Patrones sugeridos:**
- Email: `[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+`
- URL: `https?://\S+`

In [23]:
# Copie su código aca
# Columnas 'email' y 'urls' a partir del texto original (usa 'texto' o cae a 'tescto')
base = "texto" if "texto" in df_texto.columns else "tescto"
pat_email = r"[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+"
pat_url = r"https?://\S+"

df_texto["email"] = (
    df_texto[base]
    .astype(str)
    .str.findall(pat_email)
    .apply(lambda L: "; ".join(L) if L else None)
)

df_texto["urls"] = (
    df_texto[base]
    .astype(str)
    .str.findall(pat_url)
    .apply(lambda L: "; ".join(L) if L else None)
)

# Imprimir resultados
print(df_texto[[base, "email", "urls"]].to_string())


                                                                                  texto                email               urls
0    ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)                   None  https://miweb.com
1                                      Pandas > numpy? 🤔  Email: persona@example.com     persona@example.com               None
2                    Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan                 None               None
3                                  Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123                 None               None
4                                        TABLAS\t, \n espacios   y saltos de línea.\r\n                 None               None
5                              Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44                 None               None
6                                Dirección: Cll 10 # 5-20; Medellín. Barrio: La América                 

,texto,email,urls
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),[],[https://miweb.com]
1,Pandas > numpy? 🤔 Email: persona@example.com,[persona@example.com],[]
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,[],[]
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,[],[]
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",[],[]
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,[],[]
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,[],[]
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",[],[]


## 7) Hashtags, menciones y URLs en tweets

En `df_tweets`, crea las columnas `hashtags`, `mentions` y `urls`
extrayendo hashtags, menciones y URLs del texto del tweet.

In [24]:
# Copie su código aca
# Columna base del texto del tweet
base = (
    "text"
    if "text" in df_tweets.columns
    else ("tweet" if "tweet" in df_tweets.columns else "texto")
)

# Patrones
pat_hashtag = r"(?<!\w)#\w+"  # hashtags como #Café, #DataScience
pat_mention = r"(?<!\w)@\w+"  # menciones como @usuario
pat_url = r"https?://\S+"  # URLs http/https

# Crear columnas (listas de coincidencias)
df_tweets["hashtags"] = df_tweets[base].astype(str).str.findall(pat_hashtag)
df_tweets["mentions"] = df_tweets[base].astype(str).str.findall(pat_mention)
df_tweets["urls"] = df_tweets[base].astype(str).str.findall(pat_url)

# Imprimir resultados
print(df_tweets[[base, "hashtags", "mentions", "urls"]].to_string())

                                                                         tweet           hashtags         mentions                          urls
0  RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python    [#nlp, #python]         [@maria]  [http://blog.com/post?id=45]
1       ¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊  [#datos, #Python]  [@data_science]         [https://example.org]
2                             Probando cosas en Jupyter... sin link ni hashtag                 []               []                            []
3             @juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml        [#nlp, #ml]    [@juan, @ana]           [https://cursos.ai]
4                    ¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data            [#data]               []            [https://foro.com]


,tweet,hashtags,mentions,urls
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,"[#nlp, #python]",[@maria],[http://blog.com/post?id=45]
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,"[#datos, #Python]",[@data_science],[https://example.org]
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],[]
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,"[#nlp, #ml]","[@juan, @ana]",[https://cursos.ai]
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,[#data],[],[https://foro.com]


## 8) ¿Es retuit? Limpia el prefijo

Crea la columna `es_rt` (booleano) si el tweet comienza con `RT`. Luego genera la columna `tweet_sin_rt` quitando el prefijo `RT @usuario:` del inicio.

In [25]:
# Copie su código aca
# Columna base del texto del tweet
base = (
    "text"
    if "text" in df_tweets.columns
    else ("tweet" if "tweet" in df_tweets.columns else "texto")
)

# es_rt: True si comienza con "RT"
df_tweets["es_rt"] = df_tweets[base].astype(str).str.match(r"^\s*RT\b", na=False)

# tweet_sin_rt: elimina el prefijo "RT @usuario:" (soporta uno o más @menciones)
df_tweets["tweet_sin_rt"] = (
    df_tweets[base]
    .astype(str)
    .str.replace(r"^\s*RT\s+(?:@[A-Za-z0-9_]{1,15}\s*)+:\s*", "", regex=True)
)

# Imprimir resultados
print(df_tweets[[base, "es_rt", "tweet_sin_rt"]].to_string())

                                                                         tweet  es_rt                                                            tweet_sin_rt
0  RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python   True        Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python
1       ¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊  False  ¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊
2                             Probando cosas en Jupyter... sin link ni hashtag  False                        Probando cosas en Jupyter... sin link ni hashtag
3             @juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml  False        @juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml
4                    ¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data  False               ¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data


,tweet,hashtags,mentions,urls,es_rt,tweet_sin_rt
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,"[#nlp, #python]",[@maria],[http://blog.com/post?id=45],True,Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,"[#datos, #Python]",[@data_science],[https://example.org],False,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],[],False,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,"[#nlp, #ml]","[@juan, @ana]",[https://cursos.ai],False,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,[#data],[],[https://foro.com],False,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data


## 9) `tweet_limpio`

Crea `tweet_limpio` removiendo URLs, menciones y hashtags; además, elimina la puntuación sobrante, quita espacios y pasa a minúsculas.

In [26]:
# Copie su código aca
# Columna base del tweet
base = (
    "text"
    if "text" in df_tweets.columns
    else ("tweet" if "tweet" in df_tweets.columns else "texto")
)

# Crear columna 'tweet_limpio'
df_tweets["tweet_limpio"] = (
    df_tweets[base]
    .astype(str)
    .str.replace(r"(https?://\S+|www\.\S+)", " ", regex=True)  # quitar URLs
    .str.replace(r"@[A-Za-z0-9_]{1,15}", " ", regex=True)  # quitar menciones
    .str.replace(r"#\w+", " ", regex=True)  # quitar hashtags
    .str.replace(
        r"[^\w\s]", " ", regex=True
    )  # quitar puntuación/símbolos (conserva letras, números y espacios)
    .str.replace(r"_", " ", regex=True)  # quitar guión bajo sobrante
    .str.replace(r"\s+", " ", regex=True)  # colapsar espacios múltiples
    .str.strip()  # recortar extremos
    .str.lower()  # minúsculas
)

# ver resultado
print(df_tweets[[base, 'tweet_limpio']].to_string())

                                                                         tweet                                   tweet_limpio
0  RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python                       rt nuevo post en el blog
1       ¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊                              me encanta pandas
2                             Probando cosas en Jupyter... sin link ni hashtag  probando cosas en jupyter sin link ni hashtag
3             @juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml                     y lanzaron curso de nlp en
4                    ¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data                 pandas o polars debátanlo aquí


,tweet,tweet_limpio
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,nuevo post en el blog
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,me encanta pandas
2,Probando cosas en Jupyter... sin link ni hashtag,probando cosas en jupyter sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,y lanzaron curso de nlp en
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,pandas o polars debátanlo aquí


## 10) Talla en `df_productos`

Extrae y estandariza la talla:
- Letras: `XS, S, M, L, XL, XXL`, o `TU` (talla única).
- Números: captura como `talla_num`.
Crea la columna`talla_std` (letras) y `talla_num` (numérica).

In [27]:
# Copie su código aca
# Extrae y estandariza talla por letras y por números
base = "producto"

# Letras: XS, S, M, L, XL, XXL, o TU (talla única)
df_productos["talla_std"] = (
    df_productos[base].str.extract(r"(?i)\b(XXL|XL|XS|[SML])\b")[0].str.upper()
)
mask_tu = df_productos[base].str.contains(
    r"(?i)\btalla\s*(?:única|unica)\b|\b(?:única|unica)\b|\bT\.?U\.?\b|\bTU\b", na=False
)
df_productos.loc[mask_tu, "talla_std"] = "TU"

# Números: primera talla numérica que aparezca (si hay rango como 10-12 toma 10)
df_productos["talla_num"] = (
    df_productos[base]
    .str.extract(r"(?i)\b(\d{1,3}(?:[.,]\d+)?)\b")[0]  # 36, 42, 42.5, 42,5
    .str.replace(",", ".", regex=False)  # normaliza coma decimal
    .astype(float)
)

# (Opcional) ver resultados
print(df_productos[[base, "talla_std", "talla_num"]].to_string(index=False))

              producto talla_std  talla_num
        Camisa talla M         M        NaN
           Pantalón-XL        XL        NaN
     Zapato, Talla: 42       NaN       42.0
               Blusa s         S        NaN
          Medias 10-12       NaN       10.0
          Polo Talla l         L        NaN
          Vestido - 36       NaN       36.0
Sombrero (talla Única)        TU        NaN


,producto,precio,talla_std,talla_num
0,Camisa talla M,"$1.234,50",M,NaN
1,Pantalón-XL,USD 45,XL,NaN
2,"Zapato, Talla: 42","30,00 €",NaN,42.0
3,Blusa s,25.000,S,NaN
4,Medias 10-12,$ 0,NaN,10.0
5,Polo Talla l,S/. 120.90,L,NaN
6,Vestido - 36,COP 9.990,NaN,36.0
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN


## 11) Extraer de precios

Convierte la columna `precio` a numérico (`precio_num`) independiente del formato (`$1.234,50`, `USD 45`, `30,00 €`, `COP 9.990`, etc.).

In [29]:
# Copie su código aca
def _to_float_precio(v):
    if v is None:
        return np.nan
    s = re.sub(r"[^\d,.\-]", "", str(v))  # deja solo dígitos, coma, punto y signo
    if "," in s and "." in s:
        # Si hay ambos, el último separador es el decimal; el otro son miles
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "")
            s = s.replace(",", ".")
        else:
            s = s.replace(",", "")
    elif "," in s:
        # Si termina con 1–2 decimales -> coma decimal; si grupos de 3 -> miles
        if re.search(r",\d{1,2}$", s):
            s = s.replace(",", ".")
        elif re.search(r"(?:,\d{3})+$", s):
            s = s.replace(",", "")
        else:
            s = s.replace(",", ".")  # caso ambiguo, asumimos coma decimal
    elif "." in s:
        # Si termina con 1–2 decimales -> punto decimal; si grupos de 3 -> miles
        if re.search(r"(?:\.\d{3})+$", s) and not re.search(r"\.\d{1,2}$", s):
            s = s.replace(".", "")
        # else: dejamos el punto como decimal
    try:
        return float(s)
    except:
        return np.nan


df_productos["precio_num"] = df_productos["precio"].apply(_to_float_precio)

# (Opcional) ver resultados
print(df_productos[["precio", "precio_num"]].to_string(index=False))

      precio  precio_num
   $1.234,50      1234.5
      USD 45        45.0
     30,00 €        30.0
      25.000     25000.0
         $ 0         0.0
  S/. 120.90         NaN
   COP 9.990      9990.0
AR$ 2.550,00      2550.0


,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.50
1,Pantalón-XL,USD 45,XL,NaN,45.00
2,"Zapato, Talla: 42","30,00 €",NaN,42.0,30.00
3,Blusa s,25.000,S,NaN,25.00
4,Medias 10-12,$ 0,NaN,10.0,0.00
5,Polo Talla l,S/. 120.90,L,NaN,NaN
6,Vestido - 36,COP 9.990,NaN,36.0,9.99
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN,2550.00


## 12) Filtrado por texto

Filtra el DataFrame `df_productos` para mostrar solo filas cuyo `producto` contenga **camisa** o **pantalón**, ignorando acentos y mayúsculas/minúsculas.

In [30]:
# Copie su código aca
mask = (
    df_productos["producto"]
    .astype(str)
    .map(
        lambda s: "".join(
            ch
            for ch in unicodedata.normalize("NFD", s)
            if not unicodedata.combining(ch)
        )
    )  # quita acentos
    .str.lower()
    .str.contains(r"\b(camisa|pantalon)\b", na=False)
)

df_filtrado = df_productos[mask]
print(df_filtrado.to_string(index=False))

      producto    precio talla_std  talla_num  precio_num
Camisa talla M $1.234,50         M        NaN      1234.5
   Pantalón-XL    USD 45        XL        NaN        45.0


/tmp/ipykernel_21907/467063327.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  .str.contains(r"\b(camisa|pantalon)\b", na=False)


,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.5
1,Pantalón-XL,USD 45,XL,NaN,45.0


## 13) Limpieza de nombres propios

En el DataFrame `df_personas`, crea `nombre_limpio` en `df_personas`:
- Quita paréntesis y su contenido.
- Reemplaza guiones por espacio y elimina espacios extra.
- Aplica *title case* y conserva conectores (`de`, `del`, `la`, `y`) en minúscula.

In [31]:
# Copie su código aca
df_personas["nombre_limpio"] = (
    df_personas["nombre"]
    .astype(str)
    .str.replace(r"\s*\([^)]*\)", "", regex=True)  # quita ( ... )
    .str.replace("-", " ", regex=False)  # guiones -> espacio
    .str.replace(r"\s+", " ", regex=True)  # colapsa espacios múltiples
    .str.strip()
    .str.title()
    .apply(
        lambda s: " ".join(
            w.lower() if w.lower() in {"de", "del", "la", "y"} else w for w in s.split()
        )
    )
)

# ver resultado
print(df_personas[['nombre','nombre_limpio']].to_string(index=False))

               nombre     nombre_limpio
      ana María LOPEZ   Ana María Lopez
          Juan  perez        Juan Perez
        Ñandú   Gómez       Ñandú Gómez
     Miguel (Soporte)            Miguel
  MÓNICA de la CRUZ   Mónica de la Cruz
         luis-delgado      Luis Delgado


0      Ana María Lopez
1           Juan Perez
2          Ñandú Gómez
3               Miguel
4    Mónica de la Cruz
5         Luis Delgado
Name: nombre_limpio, dtype: object

## 14) Normaliza respuestas categóricas

En el DataFrame `df_personas`, convierte `categoria` en booleano `categoria_bool` donde cualquier variante de **sí** (con/ sin tilde) sea `True`; el resto `False`.

In [32]:
# Copie su código aca
df_personas["categoria_bool"] = (
    df_personas["categoria"]
    .astype(str)
    .str.contains(r"^\s*s[ií]\s*$", flags=re.I, na=False)
)

# ver resultado:
print(df_personas[['categoria','categoria_bool']].to_string(index=False))

categoria  categoria_bool
       Sí            True
       si            True
      SI             True
       No           False
        —           False
     None           False


,nombre,categoria,direccion,id_raw,nombre_limpio,categoria_bool
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001,Ana María Lopez,True
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001,Juan Perez,True
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002,Ñandú Gómez,True
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003,Miguel,False
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004,Mónica de la Cruz,False
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005,Luis Delgado,False


## 15) Ciudad desde la dirección

En el DataFrame `df_personas`, Extrae la **ciudad** en una columna `ciudad`

In [33]:
# Copie su código aca
# Extraer ciudad desde 'direccion' con separadores (coma/guion) o, en su defecto, el último bloque de texto
s = (
    df_personas["direccion"]
    .fillna("")
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

pat_sep = r"(?:,|-)\s*([A-Za-zÁÉÍÓÚáéíóúÑñÜü\. ]+)$"  # después de coma o guion
pat_tail = (
    r"([A-Za-zÁÉÍÓÚáéíóúÑñÜü\. ]+)$"  # último bloque de letras si no hay separadores
)

df_personas["ciudad"] = (
    s.str.extract(pat_sep, expand=False)
    .fillna(s.str.extract(pat_tail, expand=False))
    .str.replace(r"[.\s]+$", "", regex=True)  # limpia puntos/espacios finales
)

# ver resultado
print(df_personas[['direccion','ciudad']].to_string(index=False))

                     direccion    ciudad
      Calle 45 # 12-34, Bogotá    Bogotá
   Av. Siempre Viva 742 - Lima      Lima
     Cll. 10 No. 5-20 Medellín  Medellín
        Cra 7a # 45-60, Bogotá    Bogotá
            Av. 9 #12-34  Cali      Cali
Av. Insurgentes Sur 1234, CDMX      CDMX


,nombre,categoria,direccion,id_raw,nombre_limpio,categoria_bool,ciudad
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001,Ana María Lopez,True,Bogotá
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001,Juan Perez,True,Lima
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002,Ñandú Gómez,True,Medellín
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003,Miguel,False,Bogotá
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004,Mónica de la Cruz,False,Cali
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005,Luis Delgado,False,CDMX


## 16) Normaliza y valida IDs

En el DataFrame `df_personas`, a partir de `id_raw`, crea la columna `id_norm` con formato `ABC-0001` (tres letras + guion + 4 dígitos) y `id_valido` (True/False).

In [36]:
# Copie su código aca
df_personas["id_norm"] = df_personas["id_raw"].apply(
    lambda s: (
        lambda L, D: (
            f"{L.upper()}-{str(int(D)).zfill(4)}" if len(L) == 3 and D != "" else None
        )
    )(
        "".join(re.findall(r"[A-Za-z]", str(s)))[:3],
        (re.findall(r"\d+", str(s)) or [""])[-1],
    )
)

# Validez: exactamente 3 letras mayúsculas, guion y 4 dígitos
df_personas["id_valido"] = df_personas["id_norm"].str.match(
    r"^[A-Z]{3}-\d{4}$", na=False
)
print(df_personas[["id_raw", "id_norm", "id_valido"]].to_string())


      id_raw   id_norm  id_valido
0   abc-0001  ABC-0001       True
1    abc 001  ABC-0001       True
2    ABC0002  ABC-0002       True
3    Abc_003  ABC-0003       True
4  abc-00004  ABC-0004       True
5   ABC-0005  ABC-0005       True


,id_raw,id_norm,id_valido
0,abc-0001,ABC-0001,True
1,abc 001,ABC-0001,True
2,ABC0002,ABC-0002,True
3,Abc_003,ABC-0003,True
4,abc-00004,ABC-0004,True
5,ABC-0005,ABC-0005,True


## 17) Mini *pipeline* de limpieza general

Escribe una función `limpia_basica(s)` que: ponga el texto en minuscula, quite acentos, URLs, menciones y hashtags, puntuación, dígitos y colapse espacios. Aplícala a `df_texto['texto']` y guarda en `texto_limpio_final`.

In [38]:
# Copie su código aca
def limpia_basica(s):
    if pd.isna(s):
        return s
    s = str(s).lower()
    # quitar URLs, menciones y hashtags
    s = re.sub(r"(https?://\S+|www\.\S+)", " ", s)
    s = re.sub(r"@[A-Za-z0-9_]{1,15}", " ", s)
    s = re.sub(r"#\w+", " ", s)
    # quitar acentos/diacríticos
    s = "".join(
        ch for ch in unicodedata.normalize("NFD", s) if not unicodedata.combining(ch)
    )
    # quitar puntuación, símbolos y dígitos (dejar solo letras y espacios)
    s = re.sub(r"[^a-z\s]", " ", s)
    # colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()
    return s


df_texto["texto_limpio_final"] = df_texto["texto"].apply(limpia_basica)
print(df_texto["texto_limpio_final"].to_string())

0        hola mundo esto es un ejemplo visita
1              pandas numpy email persona com
2    me gusta el cafe colombiano es buenisimo
3       oferta x en jabon liquido ml codigo a
4           tablas espacios y saltos de linea
5                           telefono whatsapp
6    direccion cll medellin barrio la america
7                 emoji test simbolos y otros


,texto,texto_limpio_final
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),hola mundo esto es un ejemplo visita
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el cafe colombiano es buenisimo
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta x en jabon liquido ml codigo a
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",tablas espacios y saltos de linea
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,telefono whatsapp
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,direccion cll medellin barrio la america
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros


## 18)¿Qué tanto cambió el texto?

En el dataframe `df_texto`, 

Cual es la diferencia entre la longitud original (`len_raw`) y la longitud tras la limpieza (`len_clean`)?

In [39]:
# Copie su código aca
base = "texto" if "texto" in df_texto.columns else "tescto"
clean = "texto_limpio_final" if "texto_limpio_final" in df_texto.columns else base

df_texto["len_raw"] = df_texto[base].fillna("").astype(str).str.len()
df_texto["len_clean"] = df_texto[clean].fillna("").astype(str).str.len()
df_texto["diff_len"] = df_texto["len_raw"] - df_texto["len_clean"]

# Imprimir resultado (texto, limpio, longitudes y diferencia)
print(df_texto[[base, clean, "len_raw", "len_clean", "diff_len"]].to_string())

                                                                                  texto                        texto_limpio_final  len_raw  len_clean  diff_len
0    ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)        hola mundo esto es un ejemplo visita       84         36        48
1                                      Pandas > numpy? 🤔  Email: persona@example.com               pandas numpy email persona com       48         30        18
2                    Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan  me gusta el cafe colombiano es buenisimo       66         40        26
3                                  Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123     oferta x en jabon liquido ml codigo a       52         37        15
4                                        TABLAS\t, \n espacios   y saltos de línea.\r\n         tablas espacios y saltos de linea       48         33        15
5                              Teléfono:

,texto,texto_limpio_final,len_raw,len_clean
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),hola mundo esto es un ejemplo visita,84,36
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com,48,30
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el cafe colombiano es buenisimo,66,40
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta x en jabon liquido ml codigo a,52,37
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",tablas espacios y saltos de linea,48,33
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,telefono whatsapp,56,17
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,direccion cll medellin barrio la america,54,40
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros,42,27


### Guardar de resultados

Guarda el dataframe df_texto en parquet con `df_texto.to_parquet('archivo.parquet', index=False)`.

In [41]:
try:
    df_texto.to_parquet("archivo.parquet", index=False, engine="pyarrow")
except Exception:
    try:
        df_texto.to_parquet("archivo.parquet", index=False, engine="fastparquet")
    except Exception as e:
        print(
            "Instala un motor Parquet:\n  %pip install pyarrow   # o fastparquet\nError:",
            e,
        )


Instala un motor Parquet:
  %pip install pyarrow   # o fastparquet
Error: Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.


**Phd. Jose R. Zapata**
- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)